<a href="https://colab.research.google.com/github/DianaAkolzina/scientific_paper_classifier/blob/scibert/Trainer_and_Ptorch_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, Trainer, TrainingArguments
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from google.cloud import bigquery
import numpy as np
import logging
from torch.utils.tensorboard import SummaryWriter
from transformers.trainer_utils import EvalPrediction
from transformers import EarlyStoppingCallback
from google.colab import auth


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Using device: {device}")

In [ ]:
auth.authenticate_user()
project_id = "scientific-paper-classifier"
client = bigquery.Client(project=project_id)

In [ ]:
query = """
SELECT Processed_Text,  Label
FROM data_train.processed_text_10000
"""
df = client.query(query).to_dataframe()
logging.info(f"Retrieved {len(df)} rows from BigQuery")

In [ ]:
# Data Preprocessing and Balancing
text_column = 'Processed_Text'
label_column = 'Label'

combined_df = df
combined_df.reset_index(drop=True, inplace=True)
combined_df['Label'] = combined_df['Label'].astype('category')
logging.info(f"Preprocessed data: {combined_df.shape}")

label_counts = combined_df['Label'].value_counts()
min_category_size = label_counts.min()
balanced_df = pd.concat([
    combined_df[combined_df['Label'] == label].sample(n=min_category_size, random_state=42)
    for label in combined_df['Label'].cat.categories
])
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
logging.info(f"Balanced dataset: {balanced_df.shape}")

X = balanced_df['Processed_Text']
y = balanced_df['Label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
logging.info(f"Train size: {len(X_train)}, Validation size: {len(X_val)}")

In [ ]:
# Tokenization and Dataset Creation
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased', do_lower_case=True)

max_length = 256
batch_size = 32

def tokenize_data(texts, labels):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded_dict = tokenizer.encode_plus(
                            text,
                            add_special_tokens=True,
                            max_length=max_length,
                            padding='max_length',
                            truncation=True,
                            return_attention_mask=True,
                            return_tensors='pt'
                       )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels.tolist())

    return TensorDataset(input_ids, attention_masks, labels)

train_dataset = tokenize_data(X_train, y_train)
val_dataset = tokenize_data(X_val, y_val)
logging.info(f"Tokenized data: Train dataset size: {len(train_dataset)}, Validation dataset size: {len(val_dataset)}")

train_dataloader = DataLoader(
            train_dataset,
            sampler=None,
            batch_size=batch_size
        )

val_dataloader = DataLoader(
            val_dataset,
            sampler=None,
            batch_size=batch_size
        )


In [ ]:
!pip install accelerate

In [ ]:
!pip install transformers[torch]

In [ ]:
pip install accelerate -U

In [ ]:
# Model Initialization
model = BertForSequenceClassification.from_pretrained('allenai/scibert_scivocab_uncased', num_labels=len(balanced_df['Label'].unique()))
model.to(device)
logging.info(f"Initialized SciBERT model with {model.num_parameters()} parameters")

# Fine-Tuning Parameters
epochs = 20
learning_rate = 2e-5

# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)  # L2 regularization
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
logging.info(f"Training parameters: Epochs: {epochs}, Learning rate: {learning_rate}, Batch size: {batch_size}")

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=0,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    save_total_limit=2,
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_strategy="epoch",
    eval_accumulation_steps=2,
    prediction_loss_only=True,
    report_to="all",
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=lambda pred: {"accuracy": torch.sum(pred.label_ids == pred.predictions.argmax(-1)).item()},
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.0001)]
)

# Train the model
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
# Evaluation
model.eval()
predictions, true_labels = [], []

for batch in tqdm(test_dataloader, desc='Evaluating'):
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              'labels': batch[2]}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    logits = logits.detach().cpu().numpy()
    label_ids = inputs['labels'].cpu().numpy()
    predictions.append(logits)
    true_labels.append(label_ids)

# Classification Report
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

class_names = [str(label) for label in balanced_df['Label'].unique().tolist()]

report = classification_report(flat_true_labels, flat_predictions, target_names=class_names)
logging.info(f"Classification Report:\n{report}")

In [ ]:
# Save and Load Model
model_path = 'scibert_fine_tuned_model.pth'
torch.save(model.state_dict(), model_path)
logging.info(f"Fine-tuned model saved to: {model_path}")

# Load the Fine-Tuned Model
loaded_model = BertForSequenceClassification.from_pretrained('allenai/scibert_scivocab_uncased', num_labels=len(balanced_df['Label'].unique()))
loaded_model.load_state_dict(torch.load(model_path))
loaded_model.to(device)
logging.info("Fine-tuned model loaded successfully")